In [3]:
import scipy.sparse as sps
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader
from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample

In [ ]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
#1
import numpy
import itertools
from tqdm.notebook import tqdm
# GRID SEARCH RP3BETA
best=0
config={
    "k":0,
    "a":0,
    "b":0

}
cutoff_list=[10]
from Recommenders.RP3betaCBFRecommender import RP3betaCBFRecommender
URM_all=Reader.read_train_csr(values_to_replace={0:0.2})

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)
ease_r = EASE_R_Recommender(
    URM_train=URM_train,
    verbose=False
)

TopkList=numpy.arange(100, 800, 50)
aList=numpy.arange(1e2, 1e3, 500)
for x in tqdm(itertools.product(TopkList,aList),total=int(len(TopkList)*len(aList)), position=0, leave=True):
    k=x[0]
    a=x[1]
    ease_r.fit(
        topK=k,
        l2_norm=a,
    )

    result_df, _ = evaluator_validation.evaluateRecommender(ease_r)
    if result_df["MAP"][10] > best:
        best=result_df["MAP"][10]
        config={
            "k":k,
            "l2":a,
        }
        #open text file
        text_file = open("result_experiments/grid_search/ease_r.txt", "w")

        #write string to file
        text_file.write(str(config) +str(best))

        #close file
        text_file.close()

best,config

In [ ]:
best,config